In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Startups_Employee_Reviews_Data_Cleaned.csv to Startups_Employee_Reviews_Data_Cleaned.csv


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('Startups_Employee_Reviews_Data_Cleaned.csv')

X = df[['Duration','Current_Status','Work_Life_Balance','Skill_Development','Salary_and_Benefits','Job_Security','Company_Culture','Career_Growth','Work_Satisfaction','Work_Life_Balance_Likes','Career_Growth_Likes','Work_Culture_Likes','Job_Security_Likes','Innov_and_Tech_Likes','Work_Life_Balance_Dislikes','Career_Growth_Dislikes','Salary_Growth_Dislikes','Work_Culture_Dislikes','Job_Security_Dislikes','Travel_City','Travel_Country','Travel_Abroad','Timing_Nature','Working_Mon_Fri','Working_Mon_Sat_Alt','Working_Mon_Sat','Rotational_Shift','Free_transport','Child_care','Gymnasium','Cafeteria','Work_from_home','Free_food','Team_outings','Education_assistance','Soft_skill_training','Health_insurance','Job_training']]
y = df['Overall_Satisfaction']
print(X.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

print(pd.crosstab(y_test, rfc_predict, rownames=['Actual Score'], colnames=['Predicted Score']))

(20283, 38)
(20283,)
Predicted Score    1   2    3    4     5
Actual Score                            
1                364  47   40    9    14
2                 81  58  104   12     7
3                 46  53  324  200    46
4                  7   9  156  743   386
5                 10   2   43  355  1955


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 3200, stop = 3350, num = 11)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(10, 20, num = 10)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 #'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 13.1min finished


{'n_estimators': 3260, 'max_depth': 14}


In [ ]:
rfc_optimal = RandomForestClassifier(n_estimators = 3260, max_depth = 12, max_features = 'auto')
rfc_optimal.fit(X_train,y_train)
rfc_optimal_predict = rfc_optimal.predict(X_test)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc_optimal_cv_score = cross_val_score(rfc_optimal, X, y, cv=10)

# print(pd.crosstab(y_test, rfc_optimal_predict, rownames=['Actual Score'], colnames=['Predicted Score']))

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_optimal_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_optimal_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_optimal_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_optimal_cv_score.mean())

=== Confusion Matrix ===
[[ 397   20   41    6   10]
 [  83   54  106   13    6]
 [  51   33  331  223   31]
 [   7    4  128  823  339]
 [  11    0   35  359 1960]]


=== Classification Report ===
              precision    recall  f1-score   support

           1       0.72      0.84      0.78       474
           2       0.49      0.21      0.29       262
           3       0.52      0.49      0.51       669
           4       0.58      0.63      0.60      1301
           5       0.84      0.83      0.83      2365

    accuracy                           0.70      5071
   macro avg       0.63      0.60      0.60      5071
weighted avg       0.70      0.70      0.70      5071



=== All AUC Scores ===
[0.69147363 0.70921636 0.7279448  0.7066075  0.70414201 0.71055227
 0.71153846 0.71400394 0.68441815 0.7352071 ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7095104223473637
